In [1]:
from src.manager import application_manager

In [2]:
manager = application_manager.ApplicationManager()

In [3]:
manager.apiManager.retrieve_mod("the twilight forrest")

name: the twilight forrest
url:  https://api.modrinth.com/v2/search?query=the+twilight+forrest&limit=20&index=relevance&offset=0&facets=%5B%5B%22project_type+%3A+mod%22%5D%5D
- indices :  [10] 
- scores :  [85.3]
best projects:  ['The Aether II']
redirecting to api:  search_id_tag
url:  https://api.modrinth.com/v2/projects?ids=%5B%22JD2NSu5O%22%5D
redirecting to api:  id_tag
url:  https://www.curseforge.com/api/v1/mods/search?filterText=the+twilight+forrest&gameId=432&sortField=1&pageSize=50&index=0
- indices :  [0] 
- scores :  [96.9]
best projects:  ['The Twilight Forest']
redirecting to api:  search_id_tag
url:  https://api.curseforge.com/v1/mods
redirecting to api:  id_tag


TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [1]:
import re

def remove_brackets(text):
    return re.sub(r'\[.*?\]', '', text)

In [118]:
text = "Terralit [Forge/Fabric]"

print(remove_brackets(text))

NameError: name 'remove_brackets' is not defined

In [119]:
line = "\t\tThe Aether II  "
comment = "\t\t# This is a comment  "
empty = "\t\t"

filepath = r"C:\Users\Lukas\git\Minecraft\MinecraftModpackManager\data\download_list.txt"
with open(filepath, 'r') as f:
    lines = f.readlines()


In [120]:
def parse_file(lines):
    current_categories = []
    previous_indentation = 0
    previous_line = None
    mods = {}
    intendation_steps = None
    
    def add_mod(name, categories):
        print("add mod:", name)
        mods[name] = categories.copy()
    
    for line in lines:
        line = line.rstrip() # remove trailing whitespace
        
        # TODO: more preciise way to determine indentation (separate tabs from spaces)
        indentation = len(line) - len(line.lstrip())
        line = line.strip()
        
        # ignore comments and emtpy lines
        if line.startswith("#") or line is None or line == "":
            continue
        
        # ignore comment part of not-empty lines and strip again
        line = line.split("#")[0].strip()
        
        # calculate intendation difference to previous line and normalize to 1
        diff = indentation - previous_indentation
        if intendation_steps is None:
            if diff > 0:
                intendation_steps = diff
        else:
            diff //= intendation_steps
            
        print("intendation diff:", diff, "categories:", current_categories)
        if previous_line:
            if diff > 0:
                # previous line was a category
                current_categories.append(previous_line)
                
            elif diff < 0:
                # prvious line was a mod
                add_mod(previous_line, current_categories)
                # remove |diff| number of categories
                current_categories = current_categories[:diff]
                
            elif diff == 0:
                # add mod:
                add_mod(previous_line, current_categories)
                
        previous_line = line
        previous_indentation = indentation
        
    # Process the last line of the file
    if previous_line:
        add_mod(previous_line, current_categories)
        
        
    print(mods)
            

In [140]:
def parse_download_file(lines):
    
    def parse_lines(lines):
        for line in lines:
            line = line.rstrip()  # remove trailing whitespace
            indentation = len(line) - len(line.lstrip())
            line = line.strip()

            # ignore comments and empty lines
            if line.startswith("#") or not line:
                continue

            # ignore comment part of not-empty lines and strip again
            line, *comment = line.split("#", 1)
            line = line.strip()
            comment = comment[0].strip() if comment else None

            yield line, indentation, comment
    
    def add_mod(name, categories, comment = None):
        mod_list[name] = {"categories"   : categories.copy(),
                            "comment"      : comment}

        current_dict = mod_dict
        #categories = categories or [None]
        for category in categories:
            current_dict = current_dict.setdefault("category", {})
            current_dict = current_dict.setdefault(category, {})
        current_dict = current_dict.setdefault("mods", [])
        current_dict.append({"name" : name, "comment" : comment})
                
    mod_dict = {}
    mod_list = {}
            
    current_categories = []
    indentation = 0
    line = None
    intendation_steps = None
    comment = None
                
    for next_line, next_indentation, next_comment in parse_lines(lines):
        
        # calculate intendation difference to previous line and normalize to 1
        diff = next_indentation - indentation
        if intendation_steps is None:
            if diff > 0:
                intendation_steps = diff
        else:
            diff //= intendation_steps
            
        if line:
            if diff > 0:
                # previous line was a category
                current_categories.append(line)
                
            elif diff < 0:
                # prvious line was a mod
                add_mod(line, current_categories, comment)
                # remove |diff| number of categories
                current_categories = current_categories[:diff]
                
            elif diff == 0:
                # add mod:
                add_mod(line, current_categories, comment)
                
        line = next_line
        indentation = next_indentation
        comment = next_comment
        
    # Process the last line of the file
    if line:
        add_mod(line, current_categories, next_comment)
        
    return mod_dict, mod_list

In [141]:
categories2 = ["a", "b", "c"]
categories1 = ["b", "b"]

def compare_categories(categories1, categories2):
    mask = [c1 == c2 for c1, c2 in zip(categories1, categories2)]
    intendation =  mask.index(False) if False in mask else sum(mask)
    
    print(intendation)
    
compare_categories(categories1, categories2)



0


In [142]:
if 0 == False:
    print("0")

0


In [145]:
from pprint import pprint
mod_dict, db = parse_download_file(lines)
pprint(mod_dict)


{'category': {'Content': {'category': {'Pack': {'mods': [{'comment': None,
                                                          'name': 'The '
                                                                  'Aether'}]}}},
              'World': {'category': {'Generation': {'mods': [{'comment': None,
                                                              'name': 'Terralith'},
                                                             {'comment': 'This '
                                                                         'is '
                                                                         'probably '
                                                                         'a '
                                                                         'data',
                                                              'name': 'Structory'},
                                                             {'comment': None,
                                         

In [146]:
def generate_download_file(mods):        
        def get_spaces(intendation):
            return " " * intendation * 4
    
        def iterate_dict(current_dict, level = 0):
            for key, value in current_dict.items():
                if key == "mods":
                    for mod in value:
                        yield level, mod["name"], mod["comment"]
                elif key == "category":
                    yield from iterate_dict(value, level)
                else:
                    yield level, key, None
                    yield from iterate_dict(value, level + 1)
                    
        
        lines = []
        for level, name, comment in iterate_dict(mods):
            line = get_spaces(level) + name
            if comment:
                line += " # " + comment
            lines.append(line)

        
        return lines

In [147]:
generate_download_file(mod_dict)

['ServerSync',
 'Content',
 '    Pack',
 '        The Aether',
 'World',
 '    General World Mod 1',
 '    General World Mod 2',
 '    Generation',
 '        Terralith',
 '        Structory # This is probably a data',
 '        Incendium',
 '    Strucutres',
 '        Pillager Outposts']

## TinyDB tests 

In [5]:
from tinydb import TinyDB, Query
from src.config import config

In [24]:
db = TinyDB(config.mod_list_file)
db.truncate()

In [25]:
data = db.all()

db.insert({"mods" : {}})
# append something to the mods idct in mod_list
db.update({"mods" : {"a" : {}}})

[1]

In [26]:
print(db.all())

[{'mods': {'a': {}}}]


In [1]:
import asyncio

async def func2():
    print(1)
    await asyncio.sleep(1)
    print(2)

async def func1():
    task = asyncio.create_task(func2())
    print("A")
    print("B")
    #await task
    
asyncio.run(func1())

RuntimeError: asyncio.run() cannot be called from a running event loop